In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from datetime import date

import pandas as pd
import requests
from workalendar.europe import Netherlands

import data_utils

%matplotlib inline
from time import time

import warnings

In [3]:
# stations to create predictions for
stations = ['Centraal Station', 'Station Zuid', 'Station Bijlmer ArenA']

#change every week
week_no = 48

In [4]:
today = pd.to_datetime("today")
today_str = str(today.year) + "-" + str(today.month) + "-" + str(today.day)
covid_url = 'https://covidtrackerapi.bsg.ox.ac.uk/api/v2/stringency/date-range/2020-09-01/' + today_str

# Main

## 1. Get data

In [6]:
t1 = time()

herkomst_2020 = data_utils.get_gvb_data ('Datalab_Reis_Herkomst_Uur_2020')
bestemming_2020 = data_utils.get_gvb_data ('Datalab_Reis_Bestemming_Uur_2020')

t2 = time()
print('Completed in %s sec.' % (str(t2 - t1)))

Completed in 41.33223915100098 sec.


In [7]:
t1 = time()

herkomst_2021 = data_utils.get_gvb_data ('Datalab_Reis_Herkomst_Uur_2021')
bestemming_2021 = data_utils.get_gvb_data ('Datalab_Reis_Bestemming_Uur_2021')

t2 = time()
print('Completed in %s sec.' % (str(t2 - t1)))

Completed in 0.9835247993469238 sec.


In [8]:
t1 = time()

bestemming_2021 = data_utils.get_gvb_data_json(bestemming_2021)

t2 = time()
print('Completed in %s sec.' % (str(t2 - t1)))

Completed in 25.17726993560791 sec.


In [9]:
t1 = time()

knmi_obs = data_utils.get_knmi_data('knmi/knmi-observations/2021/**/**/*')

t2 = time()
print('Completed in %s sec.' % (str(t2 - t1)))

Completed in 0.5947251319885254 sec.


In [10]:
t1 = time()

knmi_preds = data_utils.get_knmi_data('knmi/knmi/**/**/**/*.json.gz')

t2 = time()
print('Completed in %s sec.' % (str(t2 - t1)))

Completed in 18.770106554031372 sec.


In [11]:
covid_measures_df = data_utils.get_covid_measures()

C:\Users\Antonis\Documents\UvA\DSP\DSP-project\model\data_utils.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  measures_rawNL["date"] = measures_rawNL.apply(


In [12]:
covid_cases_deaths_df  = data_utils.get_covid_cases_deaths()

C:\Users\Antonis\Documents\UvA\DSP\DSP-project\model\data_utils.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_covid_nl['datetime'] = pd.to_datetime(df_covid_nl['dateRep']).dt.strftime('%Y-%m-%d')


In [13]:
covid_df_raw = pd.DataFrame(requests.get(url = covid_url).json()['data'])

In [14]:
holidays_data_raw = Netherlands().holidays(2019) + Netherlands().holidays(2020) + Netherlands().holidays(2021)

In [15]:
vacations_df = data_utils.get_vacations()

In [16]:
events = data_utils.get_events()

### 2. Prepare data

#### Pre-process data sources

In [17]:
herkomst = pd.concat([herkomst_2020, herkomst_2021])
bestemming = pd.concat([bestemming_2020, bestemming_2021])

In [18]:
# Cast 'AantalReizen' to int to sum up
bestemming['AantalReizen'] = bestemming['AantalReizen'].astype(int)
herkomst['AantalReizen'] = herkomst['AantalReizen'].astype(int)

# Remove all duplicates
bestemming.drop_duplicates(inplace=True)
herkomst.drop_duplicates(inplace=True)

# Group by station name because we are analysing per station
bestemming_grouped = bestemming.groupby(['Datum', 'UurgroepOmschrijving (van aankomst)', 'AankomstHalteNaam'], as_index=False)['AantalReizen'].sum()
herkomst_grouped = herkomst.groupby(['Datum', 'UurgroepOmschrijving (van vertrek)', 'VertrekHalteNaam'], as_index=False)['AantalReizen'].sum()

In [19]:
bestemming_herkomst = data_utils.merge_bestemming_herkomst(bestemming_grouped, herkomst_grouped)

In [20]:
gvb_dfs = []

for station in stations:
    gvb_dfs.append(data_utils.preprocess_gvb_data_for_modelling(bestemming_herkomst, station))

In [21]:
knmi_historical = data_utils.preprocess_knmi_data_hour(knmi_obs)

In [22]:
knmi_forecast = data_utils.preprocess_metpre_data(knmi_preds)

In [23]:
covid_df = data_utils.preprocess_covid_data(covid_df_raw)

In [24]:
holiday_df = data_utils.preprocess_holiday_data(holidays_data_raw)

#### Merge datasources

In [25]:
gvb_dfs_merged = []

for df in gvb_dfs:
    gvb_dfs_merged.append(data_utils.merge_gvb_with_datasources(df, knmi_historical, covid_df, covid_measures_df, holiday_df, vacations_df, events, covid_cases_deaths_df))

### 3. Clean data

#### Interpolate missing data

In [26]:
gvb_dfs_interpolated = []

for df in gvb_dfs_merged:
    gvb_dfs_interpolated.append(data_utils.interpolate_missing_values(df))

C:\Users\Antonis\Documents\UvA\DSP\DSP-project\model\data_utils.py:465: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  checkins_interpolator.fit(X_train, y_train)
C:\Users\Antonis\Documents\UvA\DSP\DSP-project\model\data_utils.py:477: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  checkouts_interpolator.fit(X_train, y_train)
C:\Users\Antonis\Documents\UvA\DSP\DSP-project\model\data_utils.py:465: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  checkins_interpolator.fit(X_train, y_train)
C:\Users\Antonis\Documents\UvA\DSP\DSP-project\model\data_utils.py:477: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of

In [27]:
gvb_dfs_final = []

for df in gvb_dfs_interpolated:

    df['check-ins'] = df['check-ins'].astype(int)
    df['check-outs'] = df['check-outs'].astype(int)
    df[['check-ins_week_ago', 'check-outs_week_ago']] = df.apply(lambda x: data_utils.get_crowd_last_week(df, x), axis=1, result_type="expand")

    gvb_dfs_final.append(df)

#### 4. Create model dataframes

In [28]:
# Define features and targets. This is the same for all stations at the moment.
features = ['year', 'month', 'weekday', 'hour', 'holiday', 'vacation', 'planned_event', 'temperature', 'wind_speed', 'precipitation_h','global_radiation', 'cases', 'deaths'] + list(covid_measures_df.columns.values)
# features = ['year', 'month', 'weekday', 'hour', 'holiday', 'vacation', 'planned_event', 'temperature', 'wind_speed', 'precipitation_h','global_radiation', 'stringency', 'cases', 'deaths']

targets = ['check-ins', 'check-outs']

In [29]:
# covid_measures_df.columns

In [30]:
data_splits = []

for df in gvb_dfs_final:
    df = df[['datetime'] + features + targets]

    train, validation, test = data_utils.get_train_val_test_split(df.dropna())
    data_splits.append([train, validation, test])

In [31]:
X_train_splits = []
y_train_splits = []

X_validation_splits = []
y_validation_splits = []

X_test_splits = []
y_test_splits = []

for split in data_splits:

    X_train_splits.append(split[0][features])
    y_train_splits.append(split[0][targets])

    X_validation_splits.append(split[1][features])
    y_validation_splits.append(split[1][targets])

    X_test_splits.append(split[2][features])
    y_test_splits.append(split[2][targets])

In [32]:
# Dataframes to predict cdfheck-ins and check-outs of next week
X_predict_dfs = []

for df in gvb_dfs_final:
    X_predict_dfs.append(data_utils.get_future_df(features, df, covid_df.tail(1)['stringency'][0], covid_measures_df, covid_cases_deaths_df, holiday_df, vacations_df, knmi_forecast, events))

C:\Users\Antonis\Documents\UvA\DSP\DSP-project\model\data_utils.py:610: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['peak_period'][df.hour.isin([7, 8, 17, 18])] = 1


0     65325.0
1     65325.0
2     65325.0
3     65325.0
4     65325.0
5     65325.0
6     65325.0
7     65325.0
8     65325.0
9     65325.0
10    65325.0
11    65325.0
12    65325.0
13    65325.0
14    65325.0
15    65325.0
16    65325.0
17    65325.0
18    65325.0
19    65325.0
20    65325.0
21    65325.0
22    65325.0
23    65325.0
Name: cases, dtype: float64

C:\Users\Antonis\Documents\UvA\DSP\DSP-project\model\data_utils.py:610: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['peak_period'][df.hour.isin([7, 8, 17, 18])] = 1


0     65325.0
1     65325.0
2     65325.0
3     65325.0
4     65325.0
5     65325.0
6     65325.0
7     65325.0
8     65325.0
9     65325.0
10    65325.0
11    65325.0
12    65325.0
13    65325.0
14    65325.0
15    65325.0
16    65325.0
17    65325.0
18    65325.0
19    65325.0
20    65325.0
21    65325.0
22    65325.0
23    65325.0
Name: cases, dtype: float64

C:\Users\Antonis\Documents\UvA\DSP\DSP-project\model\data_utils.py:610: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['peak_period'][df.hour.isin([7, 8, 17, 18])] = 1


0     65325.0
1     65325.0
2     65325.0
3     65325.0
4     65325.0
5     65325.0
6     65325.0
7     65325.0
8     65325.0
9     65325.0
10    65325.0
11    65325.0
12    65325.0
13    65325.0
14    65325.0
15    65325.0
16    65325.0
17    65325.0
18    65325.0
19    65325.0
20    65325.0
21    65325.0
22    65325.0
23    65325.0
Name: cases, dtype: float64

In [33]:
print(knmi_preds['tijd'].sort_values(ascending=False))

839131    1643842800
838985    1643842800
838838    1643842800
838690    1643842800
838541    1643842800
             ...    
1         1615827600
40        1615827600
78        1615827600
39        1615824000
0         1615824000
Name: tijd, Length: 839132, dtype: object


In [34]:
# X_predict_dfs[0].dropna()
mydf = X_predict_dfs[0]
print(mydf.columns[mydf.isna().any()].tolist())
# print(mydf[~mydf['AdaptationOfWorkplace'].isna()]['cases'])

[]


### 5. Create model

In [35]:
# basic_models = []
#
# for x in range(0, len(data_splits)):
#     model_basic, r_squared_basic, mae_basic, rmse_basic = data_utils.train_random_forest_regressor(X_train_splits[x], y_train_splits[x],
#                                                                                           X_validation_splits[x], y_validation_splits[x],
#                                                                                           None)
#     basic_models.append([model_basic, r_squared_basic, mae_basic, rmse_basic])

In [36]:
#### Tune (hyper-)parameters (not done because models currently do not improve with hyperparameter tuning)

In [37]:
# Specify hyperparameters, these could be station-specific. For now, default hyperparameter settings are being used.
centraal_station_hyperparameters = None
station_zuid_hyperparameters = None
station_bijlmer_arena_hyperparameters = None

hyperparameters = [centraal_station_hyperparameters,
                   station_zuid_hyperparameters,
                   station_bijlmer_arena_hyperparameters
                   ]

In [38]:
#tuned_models = []

#for x in range(0, len(data_splits)):
#    model_tuned, r_squared_tuned, mae_tuned, rmse_tuned = data_utils.train_random_forest_regressor(X_train_splits[x], y_train_splits[x],
#                                                                                          X_validation_splits[x], y_validation_splits[x], 
#                                                                                          hyperparameters[x])
#    tuned_models.append([model_tuned, r_squared_tuned, mae_tuned, rmse_tuned])

##### Improvements compared to basic model (negative is worse performance)

In [39]:
#for x in range(0, len(basic_models)):
#    print("R-squared difference", tuned_models[x][1]-basic_models[x][1])
#    print("MAE difference", tuned_models[x][2]-basic_models[x][2])
#    print("RMSE difference", tuned_models[x][3]-basic_models[x][3])

#### Train test model (including validation data)

In [40]:
test_models = []

for x in range(0, len(data_splits)):
    X_train_with_val = pd.concat([X_train_splits[x], X_validation_splits[x]])
    y_train_with_val = pd.concat([y_train_splits[x], y_validation_splits[x]])

    model_test, r_squared_test, mae_test, rmse_test = data_utils.train_random_forest_regressor(X_train_with_val, y_train_with_val,
                                                                                          X_test_splits[x], y_test_splits[x],
                                                                                          hyperparameters[x])
    test_models.append([model_test, r_squared_test, mae_test, rmse_test])

In [41]:
test_models

[[RandomForestRegressor(random_state=1),
  0.9235309694767143,
  158.02463709677423,
  291.71866798683993],
 [RandomForestRegressor(random_state=1),
  0.8777953099640305,
  83.09130252100837,
  168.7934187018523],
 [RandomForestRegressor(random_state=1),
  0.5948012148899282,
  45.516129032258064,
  78.47413291200053]]

In [42]:
data_utils.log_models(test_models, stations, features)

#### Check models on R-squared score

In [43]:
for x in range(0, len(test_models)):
    station_name = stations[x]
    r_squared = test_models[x][1]
    if r_squared < 0.7:
        warnings.warn("Model for " + station_name + " shows unexpected performance!")

<ipython-input-43-0cf3af343546>:5: UserWarning: Model for Station Bijlmer ArenA shows unexpected performance!
  warnings.warn("Model for " + station_name + " shows unexpected performance!")


#### Train final models (to make predictions)

In [44]:
final_models = []

for x in range(0, len(data_splits)):
    X_train_with_val = pd.concat([X_train_splits[x], X_validation_splits[x], X_test_splits[x]])
    y_train_with_val = pd.concat([y_train_splits[x], y_validation_splits[x], y_test_splits[x]])

    model_final = data_utils.train_random_forest_regressor(X_train_with_val, y_train_with_val, X_test_splits[x], y_test_splits[x],
                                                  hyperparameters[x])[0]
    final_models.append(model_final)

### 6. Predict and Save

In [45]:
predictions = []
for i, model in enumerate(final_models):
    prediction = data_utils.predict(model, X_predict_dfs[i].dropna())
    predictions.append(prediction)

for i, prediction in enumerate(predictions):
    if not os.path.exists('output/' + stations[i]):
        os.mkdir('output/' + stations[i])

    prediction.to_csv(
        ('output/{}/prediction_next_week_{}.csv'.format(stations[i], date.today().strftime("%b-%d-%Y"))))